In [ ]:
import logging
logging.basicConfig()
logger = logging.getLogger()
# logger.setLevel(logging.INFO)

from models.analytics.stats import StatsAnalytics
from models.analytics.base_conditions import BaseConditionsAnalytics
from models.analytics.agent_base_conditions import AgentBaseConditionsAnalytics
from models.analytics.fit_evaluation import FitEvaluationAnalytics
from definitions import BASE_MANAGER_FOLDER, BASE_CONDITIONS_FOLDER
from graphs import precision_recall_graph, score_graph, heatmap

seeds = ['s1', 's2','s3', 's4', 's5' ]

In [ ]:
stats_analytics= StatsAnalytics(f'{BASE_MANAGER_FOLDER}/base', seeds)
summary = stats_analytics.get_mean()

In [ ]:
import seaborn as sns

sns.color_palette("mako", as_cmap=True)

score_graph(summary, seeds)

metrics_columns = ['specialist_tp', 'specialist_fp', 'specialist_fn', 'specialist_tn']
metrics = stats_analytics.get_stats(metrics_columns)

precision_recall_graph(metrics, seeds)

In [ ]:
prefit_columns = ['prefit_tp','prefit_fp','prefit_fn','prefit_tn']
postfit_columns = ['postfit_tp','postfit_fp','postfit_fn','postfit_tn']
fit_evaluation_analytics = FitEvaluationAnalytics(f'{BASE_MANAGER_FOLDER}/base', seeds, prefit_columns + postfit_columns)

prefit_metrics = fit_evaluation_analytics.get_stats(prefit_columns)
postfit_metrics = fit_evaluation_analytics.get_stats(postfit_columns)

precision_recall_graph(prefit_metrics, seeds)
precision_recall_graph(postfit_metrics, seeds)

In [ ]:
import pandas as pd

base_conditions_analytics = BaseConditionsAnalytics(f'{BASE_MANAGER_FOLDER}/base', seeds)
specialist_heatmap_list = base_conditions_analytics.get_mean()

df = pd.DataFrame(specialist_heatmap_list)
specialist_heat_df = df * 1000

heatmap(specialist_heat_df.T, title=f'Specialist Performance Through Conditions X Generation {seeds}')

In [ ]:
agent_conditions = AgentBaseConditionsAnalytics(BASE_CONDITIONS_FOLDER, seeds)

for seed in seeds:
    df = agent_conditions.get_seed(seed)
    heatmap(df.data.T, title=f'Agent Performance Through Conditions X Generation {seed}')

agent_heat_df = agent_conditions.get_mean()

heatmap(agent_heat_df.T, title=f'Agent Performance Through Conditions X Generation {seeds} Mean')

In [ ]:
start = len(agent_heat_df) - len(specialist_heat_df)

agent_heat_df = agent_heat_df[start:].reset_index()
diff = agent_heat_df.subtract(specialist_heat_df)

heatmap(diff.T.abs(), title=f'Difference of Agent/Specialist Performance Through Conditions X Generation {seeds}', colorscale='temps')